In [39]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from PIL import Image

In [40]:
class SimpleNet(nn.Module):
    
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(12288, 84)
        self.fc2 = nn.Linear(84, 50)
        self.fc3 = nn.Linear(50, 2)
        
    def forward(self, x):
        x = x.view(-1, 12288)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x))
        return x

In [41]:
train_data_path = "./training_set/"
val_data_path = "./validation_set/"
test_data_path = "./test_set/"

## Transforms.

### Resize(64):
Изображения, на которых будет происходить обучение представлены во многших разрешениях, чтобы повысить производительность обработки, мы масштабируем каждое входящее изображение до разрешения 64 х 64.

### ToTensor():
Превращаем изображения в тензор.

### Normalize:
Нормализуем тензор вокруг определённого набора средних и стандартных точек отклонения.

In [42]:
img_transforms = transforms.Compose([
    transforms.Resize((64,64)),    
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225] )
    ])

In [43]:
train_data = torchvision.datasets.ImageFolder(root=train_data_path,transform=img_transforms)
val_data = torchvision.datasets.ImageFolder(root=val_data_path,transform=img_transforms)
test_data = torchvision.datasets.ImageFolder(root=test_data_path,transform=img_transforms)

In [44]:
batch_size = 128
lr = 0.001
device = 'cpu'
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle = True)
val_data_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle = True)
test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)

In [45]:
simplenet = SimpleNet()

In [46]:
optimizer = optim.Adam(simplenet.parameters(), lr=lr)

In [47]:
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device="cpu"):
    for epoch in range(epochs):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * inputs.size(0)
        training_loss /= len(train_loader.dataset)
        
        model.eval()
        num_correct = 0 
        num_examples = 0
        for batch in val_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            output = model(inputs)
            targets = targets.to(device)
            loss = loss_fn(output,targets) 
            valid_loss += loss.data.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(output), dim=1)[1], targets).view(-1)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(val_loader.dataset)

        print(f"Epoch: {epoch + 1}, Training Loss: {training_loss}, Validation Loss: {valid_loss}, Accuracy: {num_correct / num_examples}")

In [48]:
%%time
train(simplenet, optimizer,torch.nn.CrossEntropyLoss(), train_data_loader,val_data_loader, epochs=50, device=device)

<ipython-input-40-59f5e8768372>:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(self.fc3(x))
<ipython-input-47-1a561919def2>:28: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  correct = torch.eq(torch.max(F.softmax(output), dim=1)[1], targets).view(-1)


Epoch: 1, Training Loss: 0.6892675452232361, Validation Loss: 0.6574349241256714, Accuracy: 0.621
Epoch: 2, Training Loss: 0.6343324465751647, Validation Loss: 0.660327317237854, Accuracy: 0.613
Epoch: 3, Training Loss: 0.6120355310440063, Validation Loss: 0.648521062374115, Accuracy: 0.628
Epoch: 4, Training Loss: 0.5848404750823974, Validation Loss: 0.6463684635162353, Accuracy: 0.63
Epoch: 5, Training Loss: 0.5607264590263367, Validation Loss: 0.6410220942497253, Accuracy: 0.645
Epoch: 6, Training Loss: 0.5411466283798217, Validation Loss: 0.6430005655288696, Accuracy: 0.64
Epoch: 7, Training Loss: 0.5218143277168273, Validation Loss: 0.6441792917251586, Accuracy: 0.639
Epoch: 8, Training Loss: 0.5129388098716736, Validation Loss: 0.652939893245697, Accuracy: 0.641
Epoch: 9, Training Loss: 0.49121302938461303, Validation Loss: 0.653381667137146, Accuracy: 0.642
Epoch: 10, Training Loss: 0.48360399770736695, Validation Loss: 0.6530035829544067, Accuracy: 0.638
Epoch: 11, Training Los

In [50]:
labels = ["It's a cat!", "It's a dog!"]

img = Image.open("./test_set/dogs/dog.4503.JPG") 
img = img_transforms(img).to(device)


prediction = F.softmax(simplenet(img))
prediction = prediction.argmax()
print(labels[prediction])

It's a dog!


<ipython-input-40-59f5e8768372>:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(self.fc3(x))
<ipython-input-50-c130a6b8391f>:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  prediction = F.softmax(simplenet(img))
